In [ ]:
import os
import json
import pickle
import pandas as pd
import numpy as np
from sklearn.linear_model import RidgeCV, LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold, cross_val_score
from tqdm.notebook import tqdm
from rca import make_binary_scoring, make_multiclass_scoring, process_categorical, best_logistic_solver, checker

## Loading Data

We drop feature_overlap because it contains many NaNs and compo_attribs because it doesn't have a large enough vocabulary and is also a identical to a 65 of the norms in the psychNorms dataset.

In [ ]:
with open('../../data/brain_behav_union.pkl', 'rb') as f:
    brain_behav_union = pickle.load(f)

# Loading dictionary of dtype to embed
with open('../../data/dtype_to_embed.json', 'r') as f:
    dtype_to_embed = json.load(f)
    
brain_behav_names = dtype_to_embed['brain'] + dtype_to_embed['behavior']

# Pulling and standardising embeddings
embeds = {}
embeds_path = '../../data/embeds/'
for f_name in tqdm(os.listdir(embeds_path)):
    if f_name not in ['feature_overlap.csv', 'compo_attribs.csv']:  # dropping since contains many NaNs
        
        embed = pd.read_csv(embeds_path + f_name, index_col=0)
        embed_name = f_name.split('.')[0]
        
        # Subsetting to brain and behavior vocab
        embed = embed.loc[embed.index.intersection(brain_behav_union)]
        
        # Standardising
        embeds[embed_name] = (embed - embed.mean()) / embed.std()

{name: embed.shape for name, embed in embeds.items()}

In [ ]:
with open('../../data/embed_to_dtype.json', 'r') as f:
    embed_to_type = json.load(f)
embed_to_type

In [ ]:
# Loading norms
norms = pd.read_csv('../../data/psychNorms/psychNorms_processed.zip', index_col=0, compression='zip', low_memory=False)
norm_meta = pd.read_csv('../../data/psychNorms/psychNorms_metadata_processed.csv', index_col='norm')
norms

## Cross Validation

In [ ]:
# Ridge
min_ord, max_ord = -5, 5
alphas = np.logspace(
    min_ord, max_ord, max_ord - min_ord + 1
)
ridge = RidgeCV(alphas=alphas)

# Logistic hyperparameters
Cs = 1 / alphas
inner_cv = 5
penalty = 'l2'

# Scorers
binary_scoring = make_binary_scoring()
multiclass_scoring = make_multiclass_scoring()
continuous_scoring = 'r2'

# outer_cv setting 
outer_cv, n_jobs = 5, 10

# RCA
rca = []
for embed_name in tqdm(embeds.keys()):
    embed = embeds[embed_name]
    
    to_print = []
    for norm_name in tqdm(norms.columns, desc=embed_name):
        
        # Aligning data
        y = norms[norm_name].dropna()
        X, y = embed.align(y, axis=0, join='inner', copy=True) 
        
        # Checking norm dtype 
        norm_dtype = norm_meta.loc[norm_name, 'type']
        
        # Solvers, scoring, estimators
        if norm_dtype in ['binary', 'multiclass']:
            X, y = process_categorical(outer_cv, inner_cv, X, y)
            
            # may have switched form multi to bin after processing
            norm_dtype = 'binary' if len(y.unique()) == 2 else 'multiclass'
            
            # Cross validation settings for logistic regression
            solver = best_logistic_solver(X, norm_dtype)
            
            # Defining logistic regression 
            estimator = LogisticRegressionCV(
                Cs=Cs, penalty=penalty, cv=StratifiedKFold(inner_cv),
                solver=solver, n_jobs=8
            )
            scoring = binary_scoring if norm_dtype == 'binary' else multiclass_scoring
        else: # continuous
            estimator, scoring = ridge, continuous_scoring
  
        # Cross validation
        associated_embed = norm_meta.loc[norm_name, 'associated_embed']
        check = checker(embed_name, y, norm_dtype, associated_embed, outer_cv)
        if check == 'pass':
            r2s = cross_val_score(estimator, X, y, cv=outer_cv, scoring=scoring, n_jobs=n_jobs) # stratification is automatically used for classification
            r2_mean, r2_sd = r2s.mean(), r2s.std()
        else:
            r2_mean, r2_sd = np.nan, np.nan
            
        # Saving
        train_n = int(((outer_cv - 1) / outer_cv) * len(X))
        test_n = len(X) - train_n
        p = X.shape[1]
        embed_type = embed_to_type[embed_name]
        rca.append([
            embed_name, embed_type, norm_name, train_n, test_n, p, 
            r2_mean, r2_sd, check
        ])
        
        to_print.append([norm_name, train_n, r2_mean, r2_sd, check])
            
            
    to_print = pd.DataFrame(to_print, columns=['norm' , 'train_n', 'r2_mean', 'r2_sd', 'check'])
    print(to_print.sort_values('r2_mean', ascending=False).head(10))

rca = pd.DataFrame(
    rca, columns=[
        'embed', 'embed_type', 'norm', 'train_n', 'test_n', 'p', 
        'r2_mean', 'r2_sd', 'check'
    ]
)

rca.to_csv('../../data/results/rca.csv', index=False)
rca